In [2]:
import ROOT

In [3]:
dfb = ROOT.RDataFrame("bkg", "fulldataBkg.root")
dfs = ROOT.RDataFrame("sig", "fulldataSig.root")

dfb = dfb.Define("rand", "gRandom->Rndm()")
dfs = dfs.Define("rand", "gRandom->Rndm()")

# Define filters for the three sets
def split_sets(df):
    df_train = df.Filter("rand < 0.6")
    df_valid = df.Filter("rand >= 0.6 && rand < 0.8")
    df_test  = df.Filter("rand >= 0.8")
    return df_train, df_valid, df_test

# Apply the splitting
dfs_train, dfs_valid, dfs_test = split_sets(dfs)
dfb_train, dfb_valid, dfb_test = split_sets(dfb)

# Combine signal and background for each split using RDF `AsNumpy` -> then `MakeNumpyDataFrame`
def merge_and_save(dfsplit, dfbsplit, filename):
    # Combine using ROOT RDataFrame Concatenation via snapshots
    # Save separately to temporary files
    dfsplit.Snapshot("sig", "temp_sig.root")
    dfbsplit.Snapshot("bkg", "temp_bkg.root")

    # Open both trees and merge them into one file
    f_sig = ROOT.TFile("temp_sig.root")
    f_bkg = ROOT.TFile("temp_bkg.root")

    t_sig = f_sig.Get("sig")
    t_bkg = f_bkg.Get("bkg")

    fout = ROOT.TFile(filename, "RECREATE")
    t_sig_clone = t_sig.CloneTree()
    t_bkg_clone = t_bkg.CloneTree()
    fout.Write()
    fout.Close()

    f_sig.Close()
    f_bkg.Close()

# Save merged files
merge_and_save(dfs_train, dfb_train, "trainData.root")
merge_and_save(dfs_valid, dfb_valid, "validData.root")
merge_and_save(dfs_test,  dfb_test,  "testData.root")